In [34]:
#set up dependencies
import pandas as pd
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key

In [27]:
#create list of 500+ random cities

#create random coordinates
lat_list = np.random.uniform(-90.00, 90.00, 1500)
lng_list = np.random.uniform(-180.00, 180.00, 1500)

#combine list to create coodinates
coords_list = list(zip(lat_list, lng_list))

In [32]:
#set up lists for finding cities using citipy
cities = []
countries = []

for coords in coords_list:
    
    city = citipy.nearest_city(coords[0], coords[1]).city_name
    country = citipy.nearest_city(coords[0], coords[1]).country_code
    
    if city not in cities:
        cities.append(city)

len(cities)

599

In [57]:
#perform API calls through OpenWeatherMap API

#create function to extract data from API:
def extract_data(data):
    return {
        "name" : data["name"],
        "lat" : data["coord"]["lat"],
        "lng" : data["coord"]["lon"],
        "max_temp_f" : data["main"]["temp_max"],
        "humidity" : data["main"]["humidity"],
        "cloudiness" : data["clouds"]["all"],
        "windspeed" : data["wind"]["speed"],
        "country" : data["sys"]["country"],
        "date" : data["dt"]
    }

In [59]:
#request data from OpenWeatherMay API calls

#create empty list for appending extracted data
city_results = []

#create for loop to loop over random city list
for city in cities:
    #set up request and include try/except
    try:
        data = requests.get(url = "http://api.openweathermap.org/data/2.5/weather", params = {
            "q": city,
            "appid": weather_api_key,
            "units": "imperial"
        }).json()
    
        cities_result = extract_data(data)
    
        city_results.append(cities_result)
        
    except KeyError:
        pass

In [64]:
city_df = pd.DataFrame(city_results)
city_df

,name,lat,lng,max_temp_f,humidity,cloudiness,windspeed,country,date
0,Cayenne,4.93,-52.33,87.80,66,12,12.75,GF,1600806207
1,Mataura,-46.19,168.86,53.01,93,60,8.01,NZ,1600805961
2,Cape Town,-33.93,18.42,59.00,87,0,13.87,ZA,1600806471
3,Bredasdorp,-34.53,20.04,48.20,93,0,2.24,ZA,1600805714
4,Rikitea,-23.12,-134.97,72.63,75,99,18.63,PF,1600806296
...,...,...,...,...,...,...,...,...,...
533,Broken Hill,-31.95,141.43,50.00,71,0,8.05,AU,1600806022
534,Catuday,16.29,119.81,81.63,76,89,4.50,PH,1600806024
535,Nogliki,51.83,143.17,41.74,77,0,4.16,RU,1600806585
536,Tripoli,32.88,13.19,86.95,46,23,12.84,LY,1600806585
